In [1]:
import os
import numpy as np 
import pandas as pd

import mysql.connector
from mysql.connector import Error

In [5]:
connection = mysql.connector.connect(
        host='127.0.0.1',          # Host Name
        database='sql_tutorial',   # Schemas in MySQL a.k.a Database
        user='root',               # Username
        password='****')     # Password when connect to MySQL

cursor = connection.cursor()

cursor.execute("SELECT * FROM student;") # use sql in execute() function.
cursor.fetchall() # Print all Datapoint.

# ***JOIN TABLES***

***Now, consider we want to merge two following TABLE with sql not python pandas merge***

- In Python, we can slimply use pd.merge()

In [15]:
employee_sql_df = pd.read_sql_query("SELECT * FROM employee;",connection)
employee_sql_df

,emp_id,name,birth_date,sex,salary,branch_id,sup_id
0,206,小黃,1998-10-08,F,50000,1,NaN
1,207,小綠,1995-09-16,F,29000,2,206.0
2,208,小黑,2000-05-08,F,35000,3,206.0
3,209,小白,1997-04-12,F,39000,3,207.0
4,210,小蘭,1928-11-10,F,84000,1,207.0


In [16]:
branch_sql_df = pd.read_sql_query("SELECT * FROM branch;",connection)
branch_sql_df 

,branch_id,branch_name,manager_id
0,1,研發,206.0
1,2,行政,207.0
2,3,資訊,208.0
3,4,偷懶,NaN


In [13]:
pd.merge(employee_sql_df,branch_sql_df,left_on='branch_id',right_on='branch_id',how='left')

,emp_id,name,birth_date,sex,salary,branch_id,sup_id,branch_name,manager_id
0,206,小黃,1998-10-08,F,50000,1,NaN,研發,206.0
1,207,小綠,1995-09-16,F,29000,2,206.0,行政,207.0
2,208,小黑,2000-05-08,F,35000,3,206.0,資訊,208.0
3,209,小白,1997-04-12,F,39000,3,207.0,資訊,208.0
4,210,小蘭,1928-11-10,F,84000,1,207.0,研發,206.0


- In SQL, we can do the same with following query.

In [20]:
pd.read_sql_query(
    "SELECT * FROM `employee` JOIN `branch` on `employee`.`branch_id` = `branch`.`branch_id`;"
    ,connection)

,emp_id,name,birth_date,sex,salary,branch_id,sup_id,branch_id,branch_name,manager_id
0,206,小黃,1998-10-08,F,50000,1,NaN,1,研發,206
1,210,小蘭,1928-11-10,F,84000,1,207.0,1,研發,206
2,207,小綠,1995-09-16,F,29000,2,206.0,2,行政,207
3,208,小黑,2000-05-08,F,35000,3,206.0,3,資訊,208
4,209,小白,1997-04-12,F,39000,3,207.0,3,資訊,208


- default left join

In [21]:
pd.read_sql_query(
    "SELECT * FROM `employee` LEFT JOIN `branch` on `employee`.`branch_id` = `branch`.`branch_id`;"
    ,connection)

,emp_id,name,birth_date,sex,salary,branch_id,sup_id,branch_id,branch_name,manager_id
0,206,小黃,1998-10-08,F,50000,1,NaN,1,研發,206
1,207,小綠,1995-09-16,F,29000,2,206.0,2,行政,207
2,208,小黑,2000-05-08,F,35000,3,206.0,3,資訊,208
3,209,小白,1997-04-12,F,39000,3,207.0,3,資訊,208
4,210,小蘭,1928-11-10,F,84000,1,207.0,1,研發,206


- Using right join, we can see that one branch do not have employee.

In [22]:
pd.read_sql_query(
    "SELECT * FROM `employee` RIGHT JOIN `branch` on `employee`.`branch_id` = `branch`.`branch_id`;"
    ,connection)

,emp_id,name,birth_date,sex,salary,branch_id,sup_id,branch_id,branch_name,manager_id
0,206.0,小黃,1998-10-08,F,50000.0,1.0,NaN,1,研發,206.0
1,210.0,小蘭,1928-11-10,F,84000.0,1.0,207.0,1,研發,206.0
2,207.0,小綠,1995-09-16,F,29000.0,2.0,206.0,2,行政,207.0
3,208.0,小黑,2000-05-08,F,35000.0,3.0,206.0,3,資訊,208.0
4,209.0,小白,1997-04-12,F,39000.0,3.0,207.0,3,資訊,208.0
5,NaN,None,None,None,NaN,NaN,NaN,4,偷懶,NaN


- Using INNER JOIN is the same as LEFT JOIN in this case, cause the only missing data is on the right table.

In [23]:
pd.read_sql_query(
    "SELECT * FROM `employee` INNER JOIN `branch` on `employee`.`branch_id` = `branch`.`branch_id`;"
    ,connection)

,emp_id,name,birth_date,sex,salary,branch_id,sup_id,branch_id,branch_name,manager_id
0,206,小黃,1998-10-08,F,50000,1,NaN,1,研發,206
1,210,小蘭,1928-11-10,F,84000,1,207.0,1,研發,206
2,207,小綠,1995-09-16,F,29000,2,206.0,2,行政,207
3,208,小黑,2000-05-08,F,35000,3,206.0,3,資訊,208
4,209,小白,1997-04-12,F,39000,3,207.0,3,資訊,208
